<center><h2>Подготовка работы в Colab</h2></center>

In [1]:
!pip install tqdm
from google.colab import files
src = list(files.upload().values())[0]
open('sources.zip','wb').write(src)

Saving sources.zip to sources.zip


8488758

In [0]:
import zipfile
import os
newpath = r'./data/sources' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
zip_ref = zipfile.ZipFile('sources.zip', 'r')
zip_ref.extractall('./data/sources/')
zip_ref.close()

In [0]:
!ls

data  sample_data  sources.zip


<h2><center> Подготовка данных</center></h2>

In [0]:
from os import listdir, path
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

In [4]:
SOURCE_PATH = './data/sources/'

'''
sources_ch - split by chars
'''
sources_ch = []
for file_name in tqdm(listdir(SOURCE_PATH)):
    with open(path.join(SOURCE_PATH, file_name), 'r', encoding="cp1252") as file:
        data=file.read()
        sources_ch.append([char for char in list(data)])

100%|██████████| 21941/21941 [00:02<00:00, 9042.81it/s]


<h2><center> Обработка данных</center></h2>

In [5]:

lengths_ch = [len(data) for data in sources_ch]

max_l_ch = max(lengths_ch)
min_l_ch = min(lengths_ch)

print('Максимальная длина разбинения по символам: ', max_l_ch)
print('Минимальная длина разбинения по символам: ', min_l_ch)

Максимальная длина разбинения по символам:  1023
Минимальная длина разбинения по символам:  7


In [0]:
PAD_SYMBOL = ' '
flat_sources_ch = [ch for source in sources_ch for ch in source]
flat_sources_ch.append(PAD_SYMBOL)
unique_ch = set(flat_sources_ch)
ch2index = {ch: i for (i, ch) in enumerate(unique_ch)}
index2ch = {i: ch for (i, ch) in enumerate(unique_ch)}

In [0]:
a = [ch2index[ch] for source in sources_ch for ch in source]
ch_encoder = OneHotEncoder()
ch_one_hot = ch_encoder.fit_transform(np.reshape(a, (-1, 1)))

In [0]:
ch_one_hot_sources = []
end_index = 0
for l in lengths_ch:
    ch_one_hot_sources.append(ch_one_hot[end_index:end_index+l])
    end_index += l

In [0]:
def get_batch(x, batch_size):
    x_batch = np.array([seq.toarray() for seq in x[:batch_size]])
    seq_lengths = [seq.shape[0] for seq in x_batch]
    return x_batch, seq_lengths

def next_batch(x, size):
    idx = np.arange(0 , len(x))
    np.random.shuffle(idx)
    idx = idx[:size]
    x_batch = np.array([x[i].toarray() for i in idx])
    seq_lengths = [seq.shape[0] for seq in x_batch]
    return x_batch, seq_lengths

def add_padding(batch, pad_size):
    for i in range(len(batch)):
        batch[i] = np.pad(batch[i], [(0, pad_size - len(batch[i])), (0,0)],
                          mode='constant', constant_values=0)
    return np.reshape(np.vstack(batch), (-1, batch[0].shape[0], batch[0].shape[1]))

<h2><center>Построение сети RNN VAE</center></h2>

Архитектура сети: RNN(seq2vec) -> (FC + FC) -> RNN(vec2seq) 
Генеративный Автоэнкодер на основе рекурентной сети. Для более хорошего обучения необходимо увеличивать размер сети. Подбирать различные параметры, можно несколько улучшить саму архитекуру сети. 

In [0]:
tf.reset_default_graph()

ENCODER_LAYERS = [100]
DECODER_LAYERS = [100]
N_LATENT = 100
VEC_DIM = len(unique_ch)

X = tf.placeholder(tf.float32, [None, None, VEC_DIM])
Y = tf.placeholder(tf.float32, [None, None, VEC_DIM])
seq_length = tf.placeholder(tf.int32, [None])

with tf.variable_scope('encoder'):
    encoder_layers = [tf.nn.rnn_cell.GRUCell(size) for size in ENCODER_LAYERS]
    encoder_layer_cell = tf.contrib.rnn.MultiRNNCell(encoder_layers)
    encoder_out, encoder_state = tf.nn.dynamic_rnn(cell=encoder_layer_cell,
                                         inputs=X,
                                         dtype=tf.float32,
                                         sequence_length = seq_length)

with tf.variable_scope('latent_layer'):
    stacked_encoder_out = tf.reshape(encoder_out, (-1, ENCODER_LAYERS[-1]))
    mean = tf.contrib.layers.fully_connected(stacked_encoder_out, N_LATENT, activation_fn=None)
    std_dev = 0.5 * tf.contrib.layers.fully_connected(stacked_encoder_out, N_LATENT, activation_fn=None)
    epsilon = tf.random_normal((tf.shape(stacked_encoder_out)[0], N_LATENT))
    z = mean + tf.multiply(epsilon, tf.exp(std_dev))
    z_input = tf.reshape(z, [-1, tf.shape(encoder_out)[1], N_LATENT])
    
with tf.variable_scope('decoder'):
    decoder_layers = [tf.nn.rnn_cell.GRUCell(size) for size in DECODER_LAYERS]
    decoder_layer_cell = tf.contrib.rnn.MultiRNNCell(decoder_layers)
    decoder_out, decoder_state = tf.nn.dynamic_rnn(cell=decoder_layer_cell,
                                                   inputs=z_input,
                                                   dtype=tf.float32,
                                                   sequence_length = seq_length)

with tf.variable_scope('fc'):
    stacked_rnn_outputs = tf.reshape(decoder_out, (-1, DECODER_LAYERS[-1]))
    fc= tf.contrib.layers.fully_connected(stacked_rnn_outputs, VEC_DIM, activation_fn=None)
    logit = tf.reshape(fc, [-1, tf.shape(decoder_out)[1], VEC_DIM])

with tf.variable_scope('loss'):
    target = tf.argmax(Y, axis=2)
    target_weights = tf.cast(tf.sequence_mask(seq_length, tf.reduce_max(seq_length)), tf.float32)
    target_loss = tf.contrib.seq2seq.sequence_loss(logit, target, target_weights)
    latent_loss = - 0.5 * tf.reduce_mean(1 + 2.0 * std_dev - tf.square(mean) - tf.exp(2.0 * std_dev), axis=-1)
    loss = tf.reduce_mean(target_loss + latent_loss)

In [0]:
BATCH_SIZE = 2

init = tf.global_variables_initializer()
sess = tf.Session()

sess.run(init)

x_batch, batch_seq_len = next_batch(ch_one_hot_sources, BATCH_SIZE)
print(batch_seq_len)
print(x_batch.shape)
# print(x_batch[:,:10,:].shape)
# print(x_batch[1].shape)
print('---=-=')
x_batch = add_padding(x_batch, max(batch_seq_len))
print(x_batch.shape)
x_batch = x_batch[:,:10,:]
batch_seq_len = [10, 10]
print(x_batch.shape)
# train_x = np.array(ch_one_hot_sources[:2])
# dim = train_x[0].toarray()
# print(dim)
# print(train_x)
print('---')
# print(sess.run(stacked_rnn_outputs, feed_dict={X: x_batch, seq_length: batch_seq_len})[-1].shape)
# print(sess.run(mean, feed_dict={X: x_batch, seq_length: batch_seq_len}).shape)
# print(sess.run(z, feed_dict={X: x_batch, seq_length: batch_seq_len}).shape)
# print(sess.run(decoder_input, feed_dict={X: x_batch, seq_length: batch_seq_len}))
print(sess.run(decoder_input, feed_dict={X: x_batch, seq_length: batch_seq_len}).shape)
print(sess.run(encoder_out, feed_dict={X: x_batch, seq_length: batch_seq_len}).shape)
print(sess.run(z_input, feed_dict={X: x_batch, seq_length: batch_seq_len}).shape)
# print(sess.run(logit, feed_dict={decoder_input: [[[0,0,1],[1,0,0],[0,0,0]]], seq_length: [3]}).shape)
# print(sess.run(logit, feed_dict={X: x_batch, seq_length: batch_seq_len}).shape)
# print(sess.run(target, feed_dict={Y: x_batch, seq_length: batch_seq_len}))
# print(sess.run(target_weights, feed_dict={Y: x_batch, seq_length: batch_seq_len}))
# print(sess.run(target_loss, feed_dict={X: x_batch, Y: x_batch, seq_length: batch_seq_len}))
# print(sess.run(latent_loss, feed_dict={X: x_batch, Y: x_batch, seq_length: batch_seq_len}))
# print(sess.run(loss, feed_dict={X: x_batch, Y: x_batch, seq_length: batch_seq_len}))

# print(sess.run(t, feed_dict={X: x_batch, seq_length: batch_seq_len}))


[452, 127]
(2,)
---=-=
(2, 452, 113)
(2, 10, 113)
---
(2, 10, 100)
(2, 10, 100)
(2, 10, 100)


<h2><center> Обучение сети</center></h2>

In [28]:
train_size = 0.99
X_train = ch_one_hot_sources[:int(train_size*len(ch_one_hot_sources))]
X_test = np.array([seq.toarray() for seq in ch_one_hot_sources[int(train_size*len(ch_one_hot_sources)):]])
print('Train set size: ', len(X_train))
print('Test set size: ', len(X_test))
test_seq_length = [seq.shape[0] for seq in X_test]
max_test_seq_length = max(test_seq_length)
X_test = add_padding(X_test, max_test_seq_length)

Train set size:  21721
Test set size:  220


In [29]:
LEARNING_RATE = 0.01
BATCH_SIZE = 10
EPOCHES = 3
N_BATHCES = int(len(X_train)/BATCH_SIZE)
ROUNDS = 100 # для ускорения обучения количество итераций уменьшено, сделано в целях тестирования
print('Batches: ', N_BATHCES)

optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
sess = tf.Session()

sess.run(init)

for epoch in range(EPOCHES):
    print(X_test.shape)
    test_feed_dict = {
        X: X_test,
        Y: X_test,
        seq_length: test_seq_length
    }
    test_loss = sess.run(loss, feed_dict=test_feed_dict)
    print('Epoch %u, test loss: %f' %(epoch, test_loss))
    for i in range(ROUNDS):
        x_batch, batch_seq_len = next_batch(X_train, BATCH_SIZE)
        x_batch = add_padding(x_batch, max(batch_seq_len))
#         print(x_batch.shape)
        feed_dict = {
            X: x_batch,
            Y: x_batch,
            seq_length: batch_seq_len
        }
        sess.run(training_op, feed_dict=feed_dict)
        if i % 10 == 0:
            train_loss = sess.run(loss, feed_dict=feed_dict)
            print('Epoch %u, iteration %u / %u train loss: %f' %(epoch, i, ROUNDS, train_loss))

Batches:  2172
(220, 1019, 113)
Epoch 0, test loss: 4.760016
Epoch 0, iteration 0 / 100 train loss: 4.527296
Epoch 0, iteration 10 / 100 train loss: 3.535905
Epoch 0, iteration 20 / 100 train loss: 3.598868
Epoch 0, iteration 30 / 100 train loss: 2.627381
Epoch 0, iteration 40 / 100 train loss: 1.274490
Epoch 0, iteration 50 / 100 train loss: 0.581737
Epoch 0, iteration 60 / 100 train loss: 0.342175
Epoch 0, iteration 70 / 100 train loss: 0.207530
Epoch 0, iteration 80 / 100 train loss: 0.178387
Epoch 0, iteration 90 / 100 train loss: 0.131359
(220, 1019, 113)
Epoch 1, test loss: 0.115837
Epoch 1, iteration 0 / 100 train loss: 0.142727
Epoch 1, iteration 10 / 100 train loss: 0.114833
Epoch 1, iteration 20 / 100 train loss: 0.122539
Epoch 1, iteration 30 / 100 train loss: 0.121513
Epoch 1, iteration 40 / 100 train loss: 0.102843
Epoch 1, iteration 50 / 100 train loss: 0.111844
Epoch 1, iteration 60 / 100 train loss: 0.133448
Epoch 1, iteration 70 / 100 train loss: 0.106498
Epoch 1, iter

Проверка работы сети

 Для уменьшения времени обучения было сильно занижено количество раундов обучения, уменьшена архитектура сети. Но и на данном этапе видно что loss уменьшается, сеть обучается корректно. Для более лучших результатов необходимо дальньйшее подстраввание параметров/архитекутры сети. Использование embeddings, токенизация исходного кода (тренировачные данные), все это необходимо сделать для улучшения работы сети.

работа автоэнкодера

In [59]:
# Improvement to test: check encoding, maybe don't use ord and use char directly
x_batch, batch_seq_len = next_batch(ch_one_hot_sources, 1)
x_batch = add_padding(x_batch, max(batch_seq_len))
print('Origin source chars data:')
# print(np.argmax(x_batch, -1))
origin = [index2ch[i] for i in np.argmax(x_batch, -1)[0]]
print(''.join(map(str, origin)))
softmax = tf.nn.softmax(logit, axis=-1)
result = tf.argmax(softmax, axis=-1)
gen = sess.run(result, feed_dict={X: x_batch, seq_length: batch_seq_len})
print('Generated data:')
gen = [index2ch[i] for i in gen[0]]
print(''.join(map(str, gen)))

Origin source chars data:
/* { dg-do assemble { target aarch64_asm_sve_ok } } */
/* { dg-options "-O2 -ftree-vectorize --save-temps" } */

#define TYPE double
#include "clastb_6.c"

/* { dg-final { scan-assembler {\tclastb\td[0-9]+, p[0-7], d[0-9]+, z[0-9]+\.d} } } */

Generated data:
/* { dg-do assemble { target aarch64_lsm_sve_ok } } */
/* { dg-options "-O2 -ftree-vectorize --save-tempg" } */

#define TYPE double
#include "clastb_6.c"

/* { dg-final { scan-assembler {\tclastb\td[0-9]+, p[0-7], d[0-9]+, z[0-9]+\.d} } } */



Генерация текста

Генерация "хорошего" текста требует более детальной настройки сети и более длительного обучения, рекострукция кода происходит практически идеально, что видно выше



In [62]:
#Improvement to test: check encoding, don't use ord and use char directly to make one hot encoding
mu, sigma = 0.0, 1.0
GEN_SEQ_LEN = 100
input = np.random.normal(mu, sigma, (1, GEN_SEQ_LEN, N_LATENT))
# source = [chr(ch) for ch in gen[0]]
softmax = tf.nn.softmax(logit, axis=-1)
result = tf.argmax(softmax, axis=-1)
gen = sess.run(result, feed_dict={z_input: input, seq_length: [GEN_SEQ_LEN]})
print('Generated array from noize:')
gen

Generated array from noize:


array([[110,  66,   7,  43,   9,   9,   9,  33,   9, 111,  24,  21,  82,
         82,  34, 111,   9,   1,  17,  24,  17,  52,  61,  25, 111,  86,
         34,  44,  52,  33,  37,  75,  51,  44,  17,  10,  54,   7,   7,
        108,   9,  17,  10,  66,   9, 103,  37,  54,  10,  10,  10,  26,
         66,  10, 105, 106,   9,  54,  66,  24,  70, 101,  41,  17,  17,
          9,  66,  17,  33,   9, 101,  17,  28,   9,  75,  51,  25,  19,
         24,  10,  44, 108,  10,  19,  10, 102,  19,   9,  43,  19,  25,
         17,  28,  17,  19,   9,   9,  10,   9,  19]])